# transformer test하는 파일

In [1]:
import pymysql

# 데이터 베이스 연결하기
conn = pymysql.connect(host='1.251.203.204',
                       user='root',
                       password='kdt5',
                       db='Team4',
                       charset='utf8',
                       port=33065)

curs = conn.cursor()

# 검색 명령어 사용 
sql = "SELECT eng.text as eng_text, kor.text as kor_text FROM language_en eng join language_ko kor on eng.id = kor.id;"
curs.execute(sql)
result = curs.fetchall()
print("현재 테이블의 데이터수는 총 {}개 입니다.".format(len(result)))

현재 테이블의 데이터수는 총 31102개 입니다.


In [2]:
import pandas as pd
df = pd.read_sql(sql, conn)

# 데이터베이스 연결 종료
conn.close()

C:\Users\kdp\AppData\Local\Temp\ipykernel_24736\3967427127.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [3]:
split_idx = int(len(df) * 0.8)  # 전체 데이터 중 80%를 훈련 데이터셋으로 사용
train_df = df.iloc[:split_idx]
valid_df = df.iloc[split_idx:]

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from konlpy.tag import Hannanum
from konlpy.tag import Okt


def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for text in text_iter:
        yield token_transform[language](text[language_index[language]])


SRC_LANGUAGE = "eng"
TGT_LANGUAGE = "kor"
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # 알 수 없음 / 패딩 / 인덱스 시작 / 인덱스 끝 
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]

token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy", language="en"),
    TGT_LANGUAGE: get_tokenizer(Okt().morphs),
}
print("Token Transform:")
print(token_transform)

# vocab_transform = {} # 딕셔너리 {"eng":어쩌구, "kor":저쩌구}
train_iter = train_df.values # 현길사마가 도와준 부분 
val_iter = valid_df.values

# for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
#     # train_iter = Multi30k(split="train", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
#     vocab_transform[language] = build_vocab_from_iterator(
#         generate_tokens(train_iter, language),
#         min_freq=1, # 토큰화된 단어들의 최소 빈도수 지정 
#         specials=special_symbols, # 트랜스포머에 사용하는 특수토큰 ["<unk>", "<pad>", "<bos>", "<eos>"]
#         special_first=True, # 특수 토큰을 단어 집합의 맨 앞에 추가 
#     )

# for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
#     vocab_transform[language].set_default_index(UNK_IDX)

# print("Vocab Transform:")
# print(vocab_transform)

c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


Token Transform:
{'eng': functools.partial(<function _spacy_tokenize at 0x000002594BC2CAF0>, spacy=<spacy.lang.en.English object at 0x00000259459E6910>), 'kor': <bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x000002594D2348B0>>}


In [5]:
import torch

# # 파일에 vocab_transform 저장
# torch.save(vocab_transform, './model/vocab_transform.pth')

# 파일에서 vocab_transform 불러오기
vocab_transform = torch.load('./model/vocab_transform.pth')

In [6]:
# from torch import optim
# from torchinfo import summary

# BATCH_SIZE = 128
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print(DEVICE)

# model = Seq2SeqTransformer(
#     num_encoder_layers=6,
#     num_decoder_layers=6,
#     emb_size=8,
#     max_len=128,
#     nhead=8,
#     src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
#     tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
#     dim_feedforward=512,
# ).to(DEVICE)
# criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
# optimizer = optim.Adam(model.parameters())

# summary(model)
# # for main_name, main_module in model.named_children():
# #     print(main_name)
# #     for sub_name, sub_module in main_module.named_children():
# #         print("└", sub_name)
# #         for ssub_name, ssub_module in sub_module.named_children():
# #             print("│  └", ssub_name)
# #             for sssub_name, sssub_module in ssub_module.named_children():
# #                 print("│  │  └", sssub_name)

In [7]:
import math
import torch
from torch import nn


# 포지셔 
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# TokenEmbedding 클래스로 소스 데이터와 입력 데이터를 입력 임베딩으로 변환
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers,
        num_decoder_layers,
        emb_size,
        max_len,
        nhead,
        src_vocab_size,
        tgt_vocab_size,
        dim_feedforward,
        dropout=0.1,
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            d_model=emb_size, max_len=max_len, dropout=dropout
        )
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(
        self,
        src,
        trg,
        src_mask,
        tgt_mask,
        src_padding_mask,
        tgt_padding_mask,
        memory_key_padding_mask,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )

In [8]:
from torch import optim
from torchinfo import summary

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

model = Seq2SeqTransformer(
    num_encoder_layers=3,
    num_decoder_layers=3,
    emb_size=128,
    max_len=128,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512,
).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = optim.Adam(model.parameters())

summary(model)
# for main_name, main_module in model.named_children():
#     print(main_name)
#     for sub_name, sub_module in main_module.named_children():
#         print("└", sub_name)
#         for ssub_name, ssub_module in sub_module.named_children():
#             print("│  └", ssub_name)
#             for sssub_name, sssub_module in ssub_module.named_children():
#                 print("│  │  └", sssub_name)

cpu


c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Layer (type:depth-idx)                                                 Param #
Seq2SeqTransformer                                                     --
├─TokenEmbedding: 1-1                                                  --
│    └─Embedding: 2-1                                                  1,515,904
├─TokenEmbedding: 1-2                                                  --
│    └─Embedding: 2-2                                                  3,174,912
├─PositionalEncoding: 1-3                                              --
│    └─Dropout: 2-3                                                    --
├─Transformer: 1-4                                                     --
│    └─TransformerEncoder: 2-4                                         --
│    │    └─ModuleList: 3-1                                            594,816
│    │    └─LayerNorm: 3-2                                             256
│    └─TransformerDecoder: 2-5                                         --
│    │    └─M

In [9]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

def collator(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    )

# data_iter = Multi30k(split="valid", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collator) # data_iter가 데이터셋인듯
source_tensor, target_tensor = next(iter(dataloader)) 

print("(source, target):")
print(next(iter(val_iter)))

print("source_batch:", source_tensor.shape)
print(source_tensor)

print("target_batch:", target_tensor.shape)
print(target_tensor)

(source, target):
['And they went out quickly, and fled from the sepulchre; for they trembled and were amazed: neither said they any thing to any man; for they were afraid.'
 '여자들이 심히 놀라 떨며 나와 무덤에서 도망하고 무서워하여 아무에게 아무 말도 하지 못하더라']
source_batch: torch.Size([53, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  10,  183,   10,  ...,   10,   10,  542],
        [  23,   60,  118,  ...,  349, 1208,  768],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch: torch.Size([47, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 317,  559, 2592,  ...,    7, 1243,   51],
        [  11,  142,    9,  ..., 1217,    4,  181],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


In [10]:
def generate_square_subsequent_mask(s):
    mask = (torch.triu(torch.ones((s, s), device=DEVICE)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


target_input = target_tensor[:-1, :]
target_out = target_tensor[1:, :]

source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(
    source_tensor, target_input
)

print("source_mask:", source_mask.shape)
print(source_mask)
print("target_mask:", target_mask.shape)
print(target_mask)
print("source_padding_mask:", source_padding_mask.shape)
print(source_padding_mask)
print("target_padding_mask:", target_padding_mask.shape)
print(target_padding_mask)

source_mask: torch.Size([53, 53])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
target_mask: torch.Size([46, 46])
tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
source_padding_mask: torch.Size([128, 53])
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True]

In [11]:
# model = Seq2SeqTransformer(
#     num_encoder_layers=3,
#     num_decoder_layers=3,
#     emb_size=8,
#     max_len=128,
#     nhead=8,
#     src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
#     tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
#     dim_feedforward=512,
# ).to(DEVICE)
# optimizer = optim.Adam(model.parameters()) # , lr=0.1

checkpoint = torch.load('./model/eng2kor2.pth',  map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 모델을 평가 모드로 설정
model.eval()

Seq2SeqTransformer(
  (src_tok_emb): TokenEmbedding(
    (embedding): Embedding(11843, 128)
  )
  (tgt_tok_emb): TokenEmbedding(
    (embedding): Embedding(24804, 128)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplac

In [12]:
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    memory = model.encode(source_tensor, source_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        target_mask = generate_square_subsequent_mask(ys.size(0))
        target_mask = target_mask.type(torch.bool).to(DEVICE)

        out = model.decode(ys, memory, target_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat(
            [ys, torch.ones(1, 1).type_as(source_tensor.data).fill_(next_word)], dim=0
        )
        if next_word == EOS_IDX:
            break
    return ys


def translate(model, source_sentence):
    model.eval()
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence).view(-1, 1)
    num_tokens = source_tensor.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model, source_tensor, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX
    ).flatten()
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1]
    return " ".join(output)

In [13]:
# 태초에 하나님이 천지를 창조하시니라
output_oov = translate(model, "In the beginning God created the pasta and the pizza.") # 없는 단어
output = translate(model, "In the beginning God created the heaven and the earth.")
print(output_oov)
print(output)

예수 께서 제자 들 에게 이르시되 너희 가 떡 을 살것이 떡 을 가지사 축사 하시고
예수 께서 제자 들 에게 이르시되 너희 가 어찌하여 성전 에 들어가기가 어려우니라


In [14]:
# And God said, Let there be light: and there was light.
output = translate(model, "And God said, Let there be light: and there was light.")
print(output)

예수 께서 가라사대 너희 가 성경 에 씨 를 너희 에게 비유 를 너희 에게 비유 를 너희 에게
